In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1579220448498_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-2>

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

ss = SparkSession.builder.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
import datetime
from pyspark.sql.functions import year, month, dayofmonth,dayofweek

from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.ml import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [143]:
LA_left_lag30 = ss.read.parquet("s3a://dc-group2/LA_left_lag30")
LA_left_lag30 = LA_left_lag30.where('prev30 is not null')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [146]:
col = LA_left_lag30.columns 
col.remove('label')
from pyspark.ml.feature import VectorAssembler
va = VectorAssembler(outputCol="features", inputCols=col[-30:])
va_LA_left_lag30 = va.transform(LA_left_lag30).select("features", "label")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [150]:
df_sets = va_LA_left_lag30.randomSplit([0.8, 0.2])
df_train = df_sets[0].cache()
df_valid = df_sets[1].cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [152]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor()
rfmodel = rf.fit(df_train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [168]:
#Test data.
rfpredicts = rfmodel.transform(df_valid)
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="r2") # supports "rmse" (default), "mse", "r2", and "mae"
r2 = evaluator.evaluate(rfpredicts)
print("R2 on test data = %g" % r2)

evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse") # supports "rmse" (default), "mse", "r2", and "mae"
rmse = evaluator.evaluate(rfpredicts)
print("rmse on test data = %g" % rmse )

evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae") # supports "rmse" (default), "mse", "r2", and "mae"
mae = evaluator.evaluate(rfpredicts)
print("mae on test data = %g" % mae)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

R2 on test data = 0.714139
rmse on test data = 33.6228
mae on test data = 25.751

In [183]:
col[-30:]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['prev1', 'prev2', 'prev3', 'prev4', 'prev5', 'prev6', 'prev7', 'prev8', 'prev9', 'prev10', 'prev11', 'prev12', 'prev13', 'prev14', 'prev15', 'prev16', 'prev17', 'prev18', 'prev19', 'prev20', 'prev21', 'prev22', 'prev23', 'prev24', 'prev25', 'prev26', 'prev27', 'prev28', 'prev29', 'prev30']

In [176]:
rfmodel.featureImportances.values

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([5.77284703e-01, 2.49838686e-01, 5.18767390e-02, 6.45854578e-03,
       1.99192895e-03, 8.75694903e-03, 2.61893183e-02, 9.14167915e-03,
       4.18156314e-03, 7.89270496e-03, 3.00424150e-03, 1.47859132e-03,
       2.43310459e-03, 3.98974694e-03, 1.94787620e-02, 1.98895708e-03,
       1.62192141e-03, 1.08964079e-03, 2.96401791e-03, 2.52663410e-03,
       4.15012997e-03, 2.97988435e-03, 6.77109610e-04, 2.01949257e-04,
       1.11550920e-03, 1.35900105e-03, 1.29467302e-03, 1.24269696e-03,
       1.54390655e-03, 1.24670560e-03])

In [167]:
rfpredicts.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+------------------+
|            features|label|        prediction|
+--------------------+-----+------------------+
|[33.0,38.0,93.0,1...| 44.0| 76.79401962860183|
|[36.0,37.0,44.0,8...| 39.0|63.759648257572664|
|[37.0,43.0,35.0,3...| 42.0|63.759648257572664|
|[37.0,44.0,87.0,1...| 36.0| 66.16761652806835|
|[37.0,55.0,49.0,5...| 42.0| 64.70909050134385|
|[37.0,57.0,73.0,5...| 30.0| 67.31645217966116|
|[37.0,60.0,58.0,6...| 42.0| 65.11129264087988|
|[37.0,96.0,142.0,...| 54.0| 71.76173599166094|
|[38.0,38.0,47.0,8...| 68.0| 79.46031229565338|
|[38.0,40.0,40.0,5...| 51.0| 63.89869372321478|
|[38.0,47.0,38.0,6...| 47.0| 72.81868526215831|
|[38.0,47.0,51.0,8...| 41.0| 80.03870333840872|
|[38.0,57.0,151.0,...| 57.0| 71.70267104210235|
|[38.0,59.0,57.0,4...| 54.0| 68.92006676548745|
|[38.0,59.0,128.0,...| 38.0|  70.3000403694647|
|[39.0,37.0,55.0,6...| 83.0| 66.62658828414116|
|[39.0,47.0,41.0,3...| 56.0| 64.70909050134385|
|[39.0,52.0,78.0,1...| 43.0| 70.02903463

cross-validation and tune
--

In [187]:
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae")
cv = CrossValidator().setEstimator(rf).setEvaluator(evaluator).setNumFolds(5)
paramGrid = ParamGridBuilder()\
    .addGrid(rf.maxDepth, [2,5,10])\
    .addGrid(rf.numTrees, [10,50,100])\
    .build()
cv.setEstimatorParamMaps(paramGrid)
cvmodel = cv.fit(df_train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [190]:
#Test data.
cvpredicts = cvmodel.transform(df_valid)
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="r2") # supports "rmse" (default), "mse", "r2", and "mae"
r2 = evaluator.evaluate(cvpredicts)
print("R2 on test data = %g" % r2)

evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse") # supports "rmse" (default), "mse", "r2", and "mae"
rmse = evaluator.evaluate(cvpredicts)
print("rmse on test data = %g" % rmse )

evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae") # supports "rmse" (default), "mse", "r2", and "mae"
mae = evaluator.evaluate(cvpredicts)
print("mae on test data = %g" % mae)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

R2 on test data = 0.725713
rmse on test data = 32.9351
mae on test data = 24.6701